In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 2.0 The Manipulator Jacobian

<!-- #### Manipulator Differential Kinematics Part I: Kinematics, Velocity, and Applications -->

<!-- _By Jesse Haviland and Peter Corke_ -->
$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part I: Kinematics, Velocity, and Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[2.1 Utility Functions](#utility)

[2.2 Derivative of a Pure Rotation](#dR)
* Deriving the Manipulator Jacobian
* First Derivative of an Elementary Transform
  * Derivative of a Pure Rotation

[2.3 Derivative of a Pure Translation](#dt)
* Deriving the Manipulator Jacobian
* First Derivative of an Elementary Transform
  * Derivative of a Pure Translation

[2.4 The Manipulator Jacobian](#J)
* The Manipulator Jacobian

[2.5 Fast Manipulator Jacobian](#Jf)
* Fast Manipulator Jacobian

[2.6 Changing Jacobian Frame](#frame)
* Deriving the Manipulator Jacobian

[2.7 Robotics Toolbox Jacobians](#rtb)

[2.8 Speed Analysis](#fast)


In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

# use timeit to benchmark some methods
from timeit import default_timer as timer

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

<br>

<a id='utility'></a>

### 2.1 Utility Functions
---


In this section we will use operations on skew symmetric and augmented skew-symmetric matrices

<img src="img/skew_dark.png" alt="drawing" width="1300"/>

_Shown on the left is a vector $\bf{s}\in \mathbb{R}^3$ along with its corresponding skew symmetric matrix $\bf{S} \in \bf{so}(3) \subset \mathbb{R}^{3 \times 3}$. Shown on the right is a vector $\bf{\hat{s}}\in \mathbb{R}^6$ along with its corresponding augmented skew symmetric matrix $\bf{\hat{S}} \in \bf{se}(3) \subset \mathbb{R}^{4 \times 4}$. The skew functions $[\cdot]_\times : \mathbb{R}^3 \mapsto \bf{so}(3)$ maps a vector to a skew symmetric matrix, and $[\cdot] : \mathbb{R}^6 \mapsto \bf{se}(3)$ maps a vector to an augmented skew symmetric matrix. The inverse skew functions $\mathsf{V}_\times(\cdot) : \bf{so}(3) \mapsto \mathbb{R}^3$ maps a skew symmetric matrix to a vector and $\mathsf{V}(\cdot) : \bf{se}(3) \mapsto \mathbb{R}^6$ maps an augmented skew symmetric matrix to a vector._





In [3]:

# We can program these in Python
# These methods assume the inputs are correctly sized and numpy arrays

def vex(mat):
    '''
    Converts a 3x3 skew symmetric matric to a 3 vector
    '''

    return np.array([mat[2, 1], mat[0, 2], mat[1, 0]])

def skew(vec):
    '''
    Converts a 3 vector to a 3x3 skew symmetric matrix
    '''

    return np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])


def vexa(mat):
    '''
    Converts a 4x4 augmented skew symmetric matric to a 6 vector
    '''

    return np.array([mat[0, 3], mat[1, 3], mat[2, 3], mat[2, 1], mat[0, 2], mat[1, 0]])

def skewa(vec):
    '''
    Converts a 6 vector to a 4x4 augmented skew symmetric matrix
    '''

    return np.array([
        [0, -vec[5], vec[4], vec[0]],
        [vec[5], 0, -vec[3], vec[1]],
        [-vec[4], vec[3], 0, vec[2]],
        [0, 0, 0, 0]
    ])


In [4]:
# Test our skew and vex methods out

x = np.array([1, 2, 3])

sk_x = skew(x)

print(f"x: \n{x}")
print(f"skew form of x: \n{sk_x}")
print(f"Perform vex operation on that, we should get the original a vector back: \n{vex(sk_x)}")

x: 
[1 2 3]
skew form of x: 
[[ 0 -3  2]
 [ 3  0 -1]
 [-2  1  0]]
Perform vex operation on that, we should get the original a vector back: 
[1 2 3]


In [5]:
y = np.array([1, 2, 3, 4, 5, 6])

ska_y = skewa(y)

print(f"y: \n{y}")
print(f"augmented skew form of y: \n{ska_y}")
print(f"Perform vexa operation on that, we should get the original y vector back: \n{vexa(ska_y)}")

y: 
[1 2 3 4 5 6]
augmented skew form of y: 
[[ 0 -6  5  1]
 [ 6  0 -4  2]
 [-5  4  0  3]
 [ 0  0  0  0]]
Perform vexa operation on that, we should get the original y vector back: 
[1 2 3 4 5 6]



We also need a function to extract the translational or rotational component from an $\bf{SE}(3)$ or augmented skew-symmetric matrix

<br>

<br>

<img src="img/transform_dark.png" alt="drawing" width="700"/>


_Visualisation of a homogeneous transformation matrix (the derivatives share the form of $\ \bf{T}$, except will have a 0 instead of a 1 located at $\bf{T}_{44}$). Where the matrix $\rho(\bf{T}) \in \mathbb{R}^{3 \times 3}$ of green boxes forms the rotation component, and the vector $\tau(\bf{T}) \in \mathbb{R}^{3}$ of blue boxes form the translation component. The rotation component can be extracted through the function $\rho(\cdot) : \mathbb{R}^{4\times 4} \mapsto  \mathbb{R}^{3\times 3}$, while the translation component can be extracted through the function $\tau(\cdot) : \mathbb{R}^{4\times 4} \mapsto  \mathbb{R}^{3}$._


In [6]:
def ρ(tf):
    '''
    The method extracts the rotational component from an SE3
    '''
    return tf[:3, :3]

def τ(tf):
    '''
    The method extracts the translation component from an SE3
    '''
    return tf[:3, 3]

<br>

<a id='dR'></a>
### 2.2 Derivative of a Pure Rotation
---



The derivative of a rotation matrix with respect to the rotation angle $\theta$ is required when considering a revolute joint and
can be shown to be

\begin{align*}
    \frac{\mathrm{d}\bf{R}(\theta)}
         {\mathrm{d} \theta} 
    &= {[\bf{\hat{\omega}}]_\times} \ \bf{R} \big( \theta(t) \big)
\end{align*}

where the unit vector $\bf{\hat{\omega}}$ is the joint rotation axis.

The rotation axis $\bf{\hat{\omega}}$ can be recovered using the inverse skew operator

\begin{align*}
    \bf{\hat{\omega}} &=
    \mathsf{V}_\times
    \left(
    \frac{\mathrm{d} \bf{R}(\theta)}
         {\mathrm{d} \theta}
    \bf{R}  \big( \theta(t)\big)^\top
    \right)
\end{align*}

since $ \bf{R} \in \bf{SO}(3)$, then $\bf{R}^{-1} = \bf{R}^\top$.

For an ETS, we only need to consider the elementary rotations $\bf{R}_x$, $\bf{R}_y$, and $\bf{R}_z$ which are embedded within $\bf{SE}(3)$, as $\bf{T}_{R_{x}}$, $\bf{T}_{R_{y}}$, and $\bf{T}_{R_{z}}$ i.e. pure rotations with no translational component. We can show that the derivative of each elementary rotation with respect to a rotation angle is

\begin{align*}
    \dfrac{\mathrm{d} \bf{T}_{R_{x}}(\theta)}
          {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{x}}(\theta) = \big[ \bf{\hat{R}}_x \big] \bf{T}_{R_{x}}(\theta) \\
    \dfrac{\mathrm{d} \bf{T}_{R_{y}}(\theta)}
          {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} \bf{T}_{R_{y}}(\theta) = \big[ \bf{\hat{R}}_y \big] \bf{T}_{R_{y}}(\theta) \\
    \dfrac{\mathrm{d} \bf{T}_{R_{z}}(\theta)}
          {\mathrm{d} \theta}
    &=     
    \begin{pmatrix}
        0 & -1 & 0 & 0 \\
        1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0  
    \end{pmatrix} \bf{T}_{R_{z}}(\theta) = \big[ \bf{\hat{R}}_z \big] \bf{T}_{R_{z}}(\theta)
\end{align*}

where each of the augmented skew-symmetric matrices $[\bf{\hat{R}} ]$ above corresponds to one of the generators of $\bf{SE}(3)$ which lies in $\bf{se}(3)$, the tangent space of $\bf{SE}(3)$.
If a joints defined positive rotation is a negative rotation about the axis, , then $\bf{T}_{\bf{R}_i}(\theta)^\top$ is used to calculate the derivative.


In [7]:
# We can make Python functions which perform these derivatives

def dTRx(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the x-axis by amount θ
    '''
    # This is the [Rhat_x] matrix in the maths above
    Rhx = np.array([
        [0, 0, 0, 0],
        [0, 0, -1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_x matrix in the maths above
    Trx = smb.trotx(θ)

    if flip:
        return Rhx.T @ Trx.T
    else:
        return Rhx @ Trx

def dTRy(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the y-axis by amount θ
    '''
    # This is the [Rhat_y] matrix in the maths above
    Rhy = np.array([
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_y matrix in the maths above
    Try = smb.troty(θ)

    if flip:
        return Rhy.T @ Try.T
    else:
        return Rhy @ Try

def dTRz(θ, flip):
    '''
    Calculates the derivative of an SE3 which is a pure rotation around
    the z-axis by amount θ
    '''
    # This is the [Rhat_z] matrix in the maths above
    Rhz = np.array([
        [0, -1, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    # This is the T_R_z matrix in the maths above
    Trz = smb.trotz(θ)

    if flip:
        return Rhz.T @ Trz.T
    else:
        return Rhz @ Trz


<br>

<a id='dt'></a>
### 2.3 Derivative of a Pure Translation
---


The derivative of a homogeneous transformation matrix with respect to translation is required when considering a prismatic joint. For an ETS, these translations are embedded in $\bf{SE}(3)$ as $\bf{T}_{t_x}$, $\bf{T}_{t_y}$, and $\bf{T}_{t_z}$ which are pure translations with zero rotational component. We can show that the derivative of each elementary translation with respect to a translation is 

\begin{align*}
    \dfrac{\mathrm{d} \bf{T}_{t_x}(d)}
          {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} = [\bf{\hat{t}}_x], \\
    \dfrac{\mathrm{d} \bf{T}_{t_y}(d)}
          {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} = [\bf{\hat{t}}_y], \\
    \dfrac{\mathrm{d} \bf{T}_{t_z}(d)}
          {\mathrm{d} d}
    &=     
    \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 
    \end{pmatrix} = [\bf{\hat{t}}_z],
\end{align*}

where each of the augmented skew symmetric matrices $[ \, \bf{\hat{t}} \, ]$ above are the remaining three generators of $\bf{SE}(3)$ which lie in $\bf{se}(3)$.
If a joints defined positive translation is a negative translation along the axis, 
then $-[ \, \bf{\hat{t}} \, ]$ should be used to calculate the derivative.


In [8]:
# We can make Python functions which perform these derivatives
# Since these are constant, we don't need to handle joints being
# flipped like we do with revolute joints

def dTtx():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the x-axis by amount d
    '''
    # This is the [That_x] matrix in the maths above
    Thx = np.array([
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thx

def dTty():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the y-axis by amount d
    '''
    # This is the [That_y] matrix in the maths above
    Thy = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0]
    ])

    return Thy

def dTtz():
    '''
    Calculates the derivative of an SE3 which is a pure translation along
    the z-axis by amount d
    '''
    # This is the [That_z] matrix in the maths above
    Thz = np.array([
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0]
    ])

    return Thz

<br>

<a id='J'></a>
### 2.4 The Manipulator Jacobian
---


In [9]:
# Before going any further, lets remake the panda model we made in the first notebook
E1 = rtb.ET.tz(0.333) 
E2 = rtb.ET.Rz()
E3 = rtb.ET.Ry() 
E4 = rtb.ET.tz(0.316) 
E5 = rtb.ET.Rz()
E6 = rtb.ET.tx(0.0825) 
E7 = rtb.ET.Ry(flip=True) 
E8 = rtb.ET.tx(-0.0825) 
E9 = rtb.ET.tz(0.384) 
E10 = rtb.ET.Rz()
E11 = rtb.ET.Ry(flip=True)
E12 = rtb.ET.tx(0.088) 
E13 = rtb.ET.Rx(np.pi / 2) 
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()
panda = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15

# And make a joint coordinate array q
q = np.array([0, -0.3, 0, -2.2, 0, 2, 0.79])


Now we can calculate the derivative of an ETS. To find out how the $j^{th}$ joint affects the end-effector pose, apply the chain rule to (1 in paper)

\begin{align*}
    \frac{\partial \bf{T}(\bf{q})}
         {\partial q_j} 
    &=
    \frac{\partial} 
         {\partial q_j} 
    \left(
        \bf{E}_1(\eta_1) \bf{E}_2(\eta_2) \cdot\cdot\cdot \bf{E}_M(\eta_M) 
    \right)\\
    &= 
    \prod_{i=1}^{\mu(j)-1} \!\!\!\bf{E}_i(\eta_i) 
    \ \ 
    \frac{\mathrm{d} \bf{E}_{\mu(j)}(q_j)} 
         {\mathrm{d} q_j}
    \prod_{i=\mu(j)+1}^{M} \!\!\!\!\!\!\bf{E}_i(\eta_i).
\end{align*}

The derivative of the elementary transform with respect to a joint coordinate above is obtained using one of the functions in 2.2 for a revolute joint, or 2.3 for a prismatic joint.


In [10]:

def dET(et, η):
    '''
    This method takes an ET and returns the derivative with respect to the joint coordinate
    This is here for convenience
    '''

    if et.axis == 'Rx':
        return dTRx(η, et.isflip)
    elif et.axis == 'Ry':
        return dTRy(η, et.isflip)
    elif et.axis == 'Rz':
        return dTRz(η, et.isflip)
    elif et.axis == 'tx':
        return dTtx()
    elif et.axis == 'ty':
        return dTty()
    else:
        return dTtz()


In [11]:
def dT_j(ets, j, q):
    '''
    This methods calculates the dervative of an SE3 with respect to joint coordinate j
    '''

    # Allocate an identity matrix for the result
    dT = np.eye(4)

    # Find the jth variable et in the ets
    et_j = ets.joints()[j]

    # Loop overs the ETs in the robot ETS
    for et in ets:
        if et == et_j:
            # This ET is variable and corresponds to joint j
            dT = dT @ dET(et, q[et.jindex])
        elif et.isjoint:
            # This ET is a variable joint
            # Use the q array to specify the joint angle for the variable ET
            dT = dT @ et.A(q[et.jindex])
        else:
            # This ET is static
            dT = dT @ et.A()

    return dT



We can form the angular velocity component of the $j^{th}$ column of the manipulator Jacobian

\begin{align*}
    \bf{J}_{\omega_j}(\bf{q}) 
    &=
    \mathsf{V}_\times
    \left(
        \rho
        \left(
            \frac{\partial \bf{T}(\bf{q})}
                {\partial q_j} 
        \right)
        \rho
        \left(
            \bf{T}(\bf{q})
        \right)^\top
    \right)
\end{align*}


In [12]:
def Jω_j(ets, j, q):
    '''
    This method calculates the rotational component of the jth column of the manipulator Jacobian
    '''

    # Calculate the forward kinematics at q
    T = ets.eval(q)

    # Calculate the derivative of T with respect to q_j
    dT = dT_j(ets, j, q)

    Jω = vex(ρ(dT) @ (ρ(T).T))

    return Jω
    

In [13]:
# Calculate the angular component of the third column of Panda manipulator Jacobian
print(Jω_j(panda, 2, q))

[-2.95520207e-01 -6.22199256e-33  9.55336489e-01]



We can form the translational velocity component of the $j^{th}$ column of the manipulator Jacobian using

\begin{align*}
    \bf{J}_{v_j}(\bf{q}) 
    &=
    \tau
    \left(
        \frac{\partial \bf{T}(\bf{q})}
            {\partial q_j} 
    \right).
\end{align*}


In [14]:
def Jv_j(ets, j, q):
    '''
    This method calculates the tranlation component of the jth column of the manipulator Jacobian
    '''

    # Calculate the derivative of T with respect to q_j
    dT = dT_j(ets, j, q)

    Jv = τ(dT)

    return Jv


In [15]:
# Calculate the translational component of the third column of Panda manipulator Jacobian
print(Jv_j(panda, 3, q))

[0.0372881  0.         0.47761099]



Stacking the translational and angular velocity components, the $j^{th}$ column of the manipulator Jacobian becomes 

\begin{equation*}
     \bf{J}_j(\bf{q}) = 
     \begin{pmatrix} 
          \bf{J}_{v_j}(\bf{q}) \\ 
          \bf{J}_{\omega_j}(\bf{q}) 
     \end{pmatrix} \in \mathbb{R}^{6}
\end{equation*}

where the full manipulator Jacobian is 

\begin{equation*}
    \bf{J}(\bf{q}) = 
    \begin{pmatrix} 
        \bf{J}_1(\bf{q}) & \cdots & \bf{J}_n(\bf{q})
    \end{pmatrix} \in \mathbb{R}^{6 \times n}.
\end{equation*}

<br>

<br>

<img src="img/jacobian_dark.png" alt="drawing" width="500"/>

_Visualisation of the Jacobian $\bf{J}(\bf{q})$ of a 7-jointed manipulator. Each column describes how the end-effector pose changes due to motion of the corresponding joint. The top three rows $\bf{J}_v$  correspond to the linear velocity of the end-effector while the bottom three rows $\bf{J}_\omega$ correspond to the angular velocity of the end-effector._


In [16]:
# By using our previously defined methods, we can now calculate the manipulator Jacobian

def jacob0(ets, q):
    '''
    This method calculates the manipulator Jacobian in the world frame
    '''

    # Allocate array for the Jacobian
    # It is a 6xn matrix
    J = np.zeros((6, ets.n))

    for j in range(ets.n):
        Jv = Jv_j(ets, j, q)
        Jω = Jω_j(ets, j, q)

        J[:3, j] = Jv
        J[3:, j] = Jω

    return J

In [17]:
# Calculate the manipulator Jacobian of the Panda in the world frame
J = jacob0(panda, q)

print(f"The manipulator Jacobian (world-frame) \nin configuration q = {q} \nis: \n{np.round(J, 2)}")

The manipulator Jacobian (world-frame) 
in configuration q = [ 0.   -0.3   0.   -2.2   0.    2.    0.79] 
is: 
[[ 0.11  0.29  0.1   0.04 -0.03 -0.01  0.  ]
 [ 0.46  0.    0.53  0.   -0.04  0.    0.  ]
 [ 0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
 [ 0.   -0.   -0.3   0.    0.95  0.   -0.  ]
 [ 0.    1.   -0.   -1.   -0.   -1.   -1.  ]
 [ 1.    0.    0.96  0.   -0.32  0.    0.  ]]


<br>
   
<a id='Jf'></a>
### 2.5 Fast Manipulator Jacobian
---


<img src="img/rotation_dark.png" alt="drawing" width="500"/>

_The two vector representation of a rotation matrix $\R \in \bf{SO}(3)$. The rotation matrix $\mathbb{R}$ describes the coordinate frame in terms of three orthogonal vectors $\bf{\hat{n}}$, $\bf{\hat{o}}$, and $\bf{\hat{a}}$ which are the axes of the rotated frame expressed in the reference coordinate frame $\bf{\hat{x}}$, $\bf{\hat{y}}$, and $\bf{\hat{z}}$. As shown above, each of the vectors $\bf{\hat{n}}$, $\bf{\hat{o}}$, and $\bf{\hat{a}}$ can be calculated using the cross product of the other two._



Calculating the manipulator Jacobian using the method in 2.4 is easy to understand, but has $\mathcal{O}(n^2)$ time complexity -- we can do better.


Expanding the above equations (26) and (27), and simplifying using $\bf{R} \bf{R}^\top = \bf{1}$ gives

\begin{align*}
    \bf{J}_{\omega_j}(\bf{q}) 
    &=
    \mathsf{V}_\times \bigg(
        \rho
        \left(
            ^0\bf{T}_j
        \right)
        \rho
        \left(
        [\bf{\hat{G}}_{\mu(j)}]
        \right)
        \left(
            \rho(^0\bf{T}_j)^\top
        \right)
    \bigg)
\end{align*}

where $^0\bf{T}_j$ represents the transform from the base frame to joint $j$, and $[\bf{\hat{G}}_{\mu(j)}]$ corresponds to one of the 6 generators from equations in 2.4 or (20 - 25) in the paper. 

In the case of a prismatic joint, $\rho(\bf{\hat{G}}_{\mu(j)})$ will be a $3 \times 3$ matrix of zeros which results in zero angular velocity. In the case of a revolute joint, the angular velocity is parallel to the axis of joint rotation.

\begin{align*}
    \bf{J}_{\omega_j}(\bf{q}) 
    &=
    \left\{
    \begin{matrix}
        \bf{\hat{n}}_j & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_x}\\
        \bf{\hat{o}}_j & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_y}\\
        \bf{\hat{a}}_j & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_z}\\
        (0, \ 0, \ 0)^\top & \text{if} \ \ \bf{E}_m = \bf{T}_{t}
    \end{matrix}
    \right.
\end{align*}

where $\begin{pmatrix} \bf{\hat{n}}_j & \bf{\hat{o}}_j & \bf{\hat{a}}_j \end{pmatrix} = \rho(^0\bf{T}_j)$ are the columns of a rotation matrix as shown in the Figure above.


Expanding (28) using (26) provides

\begin{align*}
    \bf{J}_{v_j}(\bf{q})
    &=
        \tau
        \left(
            ^0\bf{T}_j
            [\bf{\hat{G}}_{\mu(j)}]
            {^j\bf{T}}_{e}
        \right)
\end{align*}

which reduces to 

\begin{align*}
    \bf{J}_{v_j}(\bf{q}) 
    &=
    \left\{ 
    \begin{matrix}
        \bf{\hat{a}}_j y_e - \bf{\hat{o}}_j z_e & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_x}\\
        \bf{\hat{n}}_j z_e - \bf{\hat{a}}_j x_e & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_y}\\
        \bf{\hat{o}}_j x_e - \bf{\hat{y}}_j y_e & \text{if} \ \ \bf{E}_m = \bf{T}_{\bf{R}_z}\\
        \bf{\hat{n}}_j                      & \text{if} \ \ \bf{E}_m = \bf{T}_{t_x}\\
        \bf{\hat{o}}_j                      & \text{if} \ \ \bf{E}_m = \bf{T}_{t_y}\\
        \bf{\hat{a}}_j                      & \text{if} \ \ \bf{E}_m = \bf{T}_{t_z}\\
    \end{matrix}
    \right.
\end{align*}

where
$\begin{pmatrix}x_e & y_e & z_e \end{pmatrix}^\top = \tau({^j\bf{T}}_{e})$.

This simplification reduces the time complexity of computation of the manipulator Jacobian to $\mathcal{O}(n)$.


In [18]:
# Now making the efficient version in Python

def jacob0_efficient(ets, q):
    '''
    This method calculates the manipulator Jacobian in the world frame using a more efficient method
    '''
    T = ets.eval(q)

    U = np.eye(4)
    j = 0
    J = np.zeros((6, ets.n))

    for ets in ets:
        jindex = ets.jindex

        if ets.isjoint:
            U = U @ ets.A(q[jindex])  # type: ignore

            Tu = sm.SE3(U, check=False).inv().A @ T
            n = U[:3, 0]
            o = U[:3, 1]
            a = U[:3, 2]
            x = Tu[0, 3]
            y = Tu[1, 3]
            z = Tu[2, 3]

            if ets.isflip:
                sign = -1.0
            else:
                sign = 1.0

            if ets.axis == "Rz":
                J[:3, j] = sign * ((o * x) - (n * y))
                J[3:, j] = sign * a

            elif ets.axis == "Ry":
                J[:3, j] = sign * ((n * z) - (a * x))
                J[3:, j] = sign * o

            elif ets.axis == "Rx":
                J[:3, j] = sign * ((a * y) - (o * z))
                J[3:, j] = sign * n

            elif ets.axis == "tx":
                J[:3, j] = n
                J[3:, j] = np.array([0, 0, 0])

            elif ets.axis == "ty":
                J[:3, j] = o
                J[3:, j] = np.array([0, 0, 0])

            elif ets.axis == "tz":
                J[:3, j] = a
                J[3:, j] = np.array([0, 0, 0])

            j += 1
        else:
            U = U @ ets.A()

    return J

In [19]:
J = jacob0_efficient(panda, q)

print(f"The manipulator Jacobian (world-frame) \nin configuration q = {q} \nis: \n{np.round(J, 2)}")

The manipulator Jacobian (world-frame) 
in configuration q = [ 0.   -0.3   0.   -2.2   0.    2.    0.79] 
is: 
[[ 0.11  0.29  0.1   0.04 -0.03 -0.01 -0.  ]
 [ 0.46  0.    0.53 -0.   -0.04 -0.    0.  ]
 [ 0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
 [ 0.    0.   -0.3  -0.    0.95 -0.   -0.  ]
 [ 0.    1.    0.   -1.    0.   -1.   -1.  ]
 [ 1.    0.    0.96 -0.   -0.32 -0.    0.  ]]


<br>

<a id='frame'></a>
### 2.6 Changing Jacobian Frame
---


We have been calculating the manipulator Jacobian in the world-frame

\begin{equation*}
     {^0\bf{J}}(\bf{q}).
\end{equation*}

is the manipulator Jacobian matrix expressed in the world-coordinate frame.

The Jacobian expressed in the end-effector frame is

\begin{equation*}
    {^e\bf{J}}(\bf{q}) =
    \begin{pmatrix} 
        {{^0\bf{R}}_e^\top} & \bf{0} \\ 
        \bf{0} & {{^0\bf{R}}_e^\top}
     \end{pmatrix}
     {^0\bf{J}}(\bf{q})
\end{equation*}

where ${^0\bf{R}}_e$ is a rotation matrix representing the orientation of the end-effector in the world frame.



In [20]:
# Now making it in Python

def jacobe(ets, q):
    '''
    This method calculates the manipulator Jacobian in the end-effector frame
    '''

    # Calculate world-frame Jacobian
    J0 = jacob0_efficient(ets, q)

    # Calculate the forward kineamtics
    T = ets.eval(q)

    # Extract rotation from T and transpose
    R = T[:3, :3].T

    # Make our velocity transform matrix
    tr = np.zeros((6, 6))

    # Put the rotation in the top left and bottom right of the matrix
    tr[:3, :3] = R
    tr[3:, 3:] = R

    # Perform the matrix multiplication to calculate the Jacobian
    # in the end-effector frame
    Je = tr @ J0

    return Je
    

In [21]:
# Calculate the manipulator Jacobian of the Panda in the end-effector frame
J = jacobe(panda, q)

print(f"The manipulator Jacobian (end-effector) \nin configuration q = {q} \nis: \n{np.round(J, 2)}")

The manipulator Jacobian (end-effector) 
in configuration q = [ 0.   -0.3   0.   -2.2   0.    2.    0.79] 
is: 
[[ 0.07 -0.18  0.09  0.39 -0.1   0.06  0.  ]
 [-0.08 -0.52 -0.06  0.27 -0.04  0.06  0.  ]
 [-0.46 -0.   -0.53  0.    0.04  0.    0.  ]
 [ 0.78  0.    0.56 -0.    0.34 -0.   -0.  ]
 [ 0.63  0.    0.83 -0.   -0.94 -0.    0.  ]
 [ 0.   -1.    0.    1.   -0.    1.    1.  ]]


<br>

<a id='rtb'></a>
### 2.7 Robotics Toolbox Jacobians
---



The `ETS` class within the Robotics Toolbox supports many kinematic methods (such as the previously discussed `.eval` and `.fkine` methods).

The `.jacob0` and `.jacobe` methods in the `ETS` class calculate the world and end-effector frame Jacobians respectively.


In [22]:
# Note: The panda object which we have been using is an instance of the ETS class

# Calculate the world frame Jacobian using the ETS class
J0 = panda.jacob0(q)

# Calculate the end-effector frame Jacobian using the ETS class
Je = panda.jacobe(q)

# View our Jacobians
print(f"The manipulator Jacobian (world frame) is: \n{np.round(J0, 2)}")
print(f"\nThe manipulator Jacobian (end-effector frame) is: \n{np.round(Je, 2)}")

The manipulator Jacobian (world frame) is: 
[[ 0.11  0.29  0.1   0.04 -0.03 -0.01  0.  ]
 [ 0.46  0.    0.53  0.   -0.04  0.    0.  ]
 [-0.   -0.46  0.03  0.48 -0.1   0.09  0.  ]
 [ 0.   -0.   -0.3   0.    0.95  0.   -0.  ]
 [ 0.    1.    0.   -1.    0.   -1.   -1.  ]
 [ 1.    0.    0.96  0.   -0.32  0.    0.  ]]

The manipulator Jacobian (end-effector frame) is: 
[[ 0.07 -0.18  0.09  0.39 -0.1   0.06  0.  ]
 [-0.08 -0.52 -0.06  0.27 -0.04  0.06  0.  ]
 [-0.46 -0.   -0.53  0.    0.04  0.    0.  ]
 [ 0.78  0.    0.56 -0.    0.34 -0.    0.  ]
 [ 0.63  0.    0.83 -0.   -0.94 -0.    0.  ]
 [ 0.   -1.    0.    1.   -0.    1.    1.  ]]


<br>

<a id='fast'></a>
### 2.8 Speed Comparison
---



In this section we will briefly cover a speed comparison between the two `jacob0` methods shown here and the Jacobian implementation provided by the Robotics Toolbox.

Kinematic methods in the toolbox are written using C extensions and wrapped by python methods. These C methods also have Python fallbacks to rely on if something goes wrong.


In [23]:
# Number of iterations to test over
n = 1000

# Make a random matrix of q values
qt = np.random.random((n, 7))

# Slow Jacobian implementation
start = timer()
for q in qt:
    jacob0(panda, q)
slow_time = timer() - start

# Improved Jacobian implementation
start = timer()
for q in qt:
    jacob0_efficient(panda, q)
improved_time = timer() - start

# Toolbox C improved Jacobian implementation
start = timer()
for q in qt:
    panda.jacob0(q)
rtb_time = timer() - start

# Make a table
print(f"\nTiming to Calculate Manipulator Jacobian over {n} iterations\n")
table = ANSITable(
    "Function",
    "Total Time (ms)",
    "Average Time (us)",
    border="thin",
)

table.row(
    "our jacob0",
    np.round(slow_time * 1000.0, 4),
    np.round((slow_time / n) * 1000000.0, 4),
)

table.row(
    "our jacob0_efficient",
    np.round(improved_time * 1000.0, 4),
    np.round((improved_time / n) * 1000000.0, 4),
)

table.row(
    "toolbox jacob0",
    np.round(rtb_time * 1000.0, 4),
    np.round((rtb_time / n) * 1000000.0, 4),
)
table.print()



Timing to Calculate Manipulator Jacobian over 1000 iterations

┌─────────────────────┬─────────────────┬───────────────────┐
│            Function │ Total Time (ms) │ Average Time (us) │
├─────────────────────┼─────────────────┼───────────────────┤
│          our jacob0 │        766.2759 │          766.2759 │
│our jacob0_efficient │         84.0204 │           84.0204 │
│      toolbox jacob0 │          0.6925 │            0.6925 │
└─────────────────────┴─────────────────┴───────────────────┘

